In [31]:
# Data Collection
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

# Load the dataset
data = gutenberg.raw('shakespeare-hamlet.txt')

#Save to a file
with open('hamlet.txt', 'w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to /home/hamid/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [32]:
# Data Preprocessing
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
with open('hamlet.txt', 'r') as file:
    text = file.read().lower()

# Tokenize the text-- Creating indexes for words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index)+1
total_words

4818

In [33]:
# Create input sequences
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[: i+1]
        input_sequences.append(n_gram_sequence)

input_sequences

[[1, 687],
 [1, 687, 4],
 [1, 687, 4, 45],
 [1, 687, 4, 45, 41],
 [1, 687, 4, 45, 41, 1886],
 [1, 687, 4, 45, 41, 1886, 1887],
 [1, 687, 4, 45, 41, 1886, 1887, 1888],
 [1180, 1889],
 [1180, 1889, 1890],
 [1180, 1889, 1890, 1891],
 [57, 407],
 [57, 407, 2],
 [57, 407, 2, 1181],
 [57, 407, 2, 1181, 177],
 [57, 407, 2, 1181, 177, 1892],
 [407, 1182],
 [407, 1182, 63],
 [408, 162],
 [408, 162, 377],
 [408, 162, 377, 21],
 [408, 162, 377, 21, 247],
 [408, 162, 377, 21, 247, 882],
 [18, 66],
 [451, 224],
 [451, 224, 248],
 [451, 224, 248, 1],
 [451, 224, 248, 1, 30],
 [408, 407],
 [451, 25],
 [408, 6],
 [408, 6, 43],
 [408, 6, 43, 62],
 [408, 6, 43, 62, 1893],
 [408, 6, 43, 62, 1893, 96],
 [408, 6, 43, 62, 1893, 96, 18],
 [408, 6, 43, 62, 1893, 96, 18, 566],
 [451, 71],
 [451, 71, 51],
 [451, 71, 51, 1894],
 [451, 71, 51, 1894, 567],
 [451, 71, 51, 1894, 567, 378],
 [451, 71, 51, 1894, 567, 378, 80],
 [451, 71, 51, 1894, 567, 378, 80, 3],
 [451, 71, 51, 1894, 567, 378, 80, 3, 273],
 [451, 71

In [34]:
# Pad sequence
max_sequence_len = max([len(x) for x in input_sequences])
max_sequence_len

14

In [35]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [36]:
# Create predictors and labels
import tensorflow as tf
x, y = input_sequences[:,:-1], input_sequences[:,-1]

In [37]:
x

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       ...,
       [   0,    0,    0, ...,  687,    4,   45],
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4]], dtype=int32)

In [38]:
y

array([ 687,    4,   45, ..., 1047,    4,  193], dtype=int32)

In [39]:
# Converting the y label into categorical
y = tf.keras.utils.to_categorical(y, num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [40]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [41]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [42]:
# Train our LSTM RNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Define the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length = max_sequence_len-1))
model.add(LSTM(150, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation ='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 13, 100)           481800    
                                                                 
 lstm_2 (LSTM)               (None, 13, 150)           150600    
                                                                 
 dropout_1 (Dropout)         (None, 13, 150)           0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               100400    
                                                                 
 dense_1 (Dense)             (None, 4818)              486618    
                                                                 
Total params: 1219418 (4.65 MB)
Trainable params: 1219418 (4.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [43]:
# Train the model
history = model.fit(x_train, 
                    y_train,
                    epochs = 250,
                    validation_data = (x_test, y_test),
                    verbose = 1
                    )

Epoch 1/250
644/644 [==============================] - 10s 12ms/step - loss: 6.9125 - accuracy: 0.0316 - val_loss: 6.7532 - val_accuracy: 0.0365
Epoch 2/250
644/644 [==============================] - 7s 12ms/step - loss: 6.4699 - accuracy: 0.0377 - val_loss: 6.8300 - val_accuracy: 0.0437
Epoch 3/250
644/644 [==============================] - 7s 11ms/step - loss: 6.3252 - accuracy: 0.0465 - val_loss: 6.8921 - val_accuracy: 0.0544
Epoch 4/250
644/644 [==============================] - 7s 11ms/step - loss: 6.1828 - accuracy: 0.0526 - val_loss: 6.9277 - val_accuracy: 0.0548
Epoch 5/250
644/644 [==============================] - 7s 11ms/step - loss: 6.0412 - accuracy: 0.0576 - val_loss: 6.9341 - val_accuracy: 0.0579
Epoch 6/250
644/644 [==============================] - 7s 11ms/step - loss: 5.8952 - accuracy: 0.0655 - val_loss: 6.9880 - val_accuracy: 0.0622
Epoch 7/250
644/644 [==============================] - 7s 11ms/step - loss: 5.7527 - accuracy: 0.0740 - val_loss: 7.0309 - val_accuracy

In [44]:
# Function to predict the next word
def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):] # Ensure the sequence length matches the max sequence legth -1
    token_list = pad_sequences([token_list], maxlen = max_sequence_len-1, padding= 'pre')
    predicted = model.predict(token_list, verbose = 0)
    predicted_word_index = np.argmax(predicted, axis = 1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [45]:
input_text = 'Honesty is the best '
print(f'Input Text: {input_text}')
max_sequence_len = model.input_shape[1]+1
next_word = predict_next_word(model, tokenizer, input_text, max_sequence_len)
print(f'Next Word Prediction: {next_word}')

Input Text: Honesty is the best 
Next Word Prediction: of


In [46]:
# Save the model
model.save('next_word_lstm.h5')

# Save the tokenizer
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

/home/hamid/Desktop/GenerativeAI/ANN/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [47]:
input_text = " Bar. 'Tis now strook twelue, get thee to bed"
print(f'Input Text: {input_text}')
max_sequence_len = model.input_shape[1]+1
next_word = predict_next_word(model, tokenizer, input_text, max_sequence_len)
print(f'Next Word Prediction: {next_word}')

Input Text:  Bar. 'Tis now strook twelue, get thee to bed
Next Word Prediction: francisco
